In [24]:
import os
from pathlib import Path
from sqlalchemy import create_engine, inspect

# 1. Get the Project Root (One folder up from 'scripts')
# Path.cwd() is ".../chicago-crime/scripts"
# .parent    is ".../chicago-crime" (The Root)
project_root = Path.cwd().parent 
db_path = project_root / "chicago-crime"/ "chicago_crime.db"

print(f"Looking for database at: {db_path}")

# 2. Check if the file actually exists before connecting
if not db_path.exists():
    print("CRITICAL: Database file not found at root! Did you run the backfill?")
else:
    print("File found. Connecting...")

# 3. Connect using the absolute path
# We use f"sqlite:///{db_path}" (3 slashes + path)
engine = create_engine(f"sqlite:///{db_path}", echo=False)

def get_db_schema() -> None:
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    
    if not tables:
        print("Connected, but the database has no tables (Empty).")
        return

    print(f"Success! Found tables: {tables}")
    for table_name in tables:
        print(f"\nTable: {table_name}")
        # Print first 3 columns just to verify
        columns = inspector.get_columns(table_name)
        for col in columns:
            print(f"  - {col['name']} ({col['type']})")

get_db_schema()

Looking for database at: c:\Users\nealv\OneDrive\Desktop\DA Materials\Study 2025\Python\Projects\chicago-crime\chicago_crime.db
File found. Connecting...
Success! Found tables: ['crime_records']

Table: crime_records
  - id (INTEGER)
  - case_number (VARCHAR(20))
  - date (DATETIME)
  - updated_on (DATETIME)
  - block (VARCHAR(100))
  - iucr (VARCHAR(10))
  - primary_type (VARCHAR(100))
  - description (VARCHAR(255))
  - location_description (VARCHAR(100))
  - arrest (BOOLEAN)
  - domestic (BOOLEAN)
  - beat (VARCHAR(10))
  - district (VARCHAR(10))
  - ward (INTEGER)
  - community_area (VARCHAR(10))
  - fbi_code (VARCHAR(10))
  - x_coordinate (FLOAT)
  - y_coordinate (FLOAT)
  - latitude (FLOAT)
  - longitude (FLOAT)
  - year (INTEGER)


In [26]:
import os
from pathlib import Path
from sqlalchemy import create_engine, inspect, text
import pandas as pd


# 1. Get the Project Root (One folder up from 'scripts')
# Path.cwd() is ".../chicago-crime/scripts"
# .parent    is ".../chicago-crime" (The Root)
project_root = Path.cwd().parent 
db_path = project_root / "chicago-crime"/ "chicago_crime.db"

print(f"Looking for database at: {db_path}")

# 2. Check if the file actually exists before connecting
if not db_path.exists():
    print("CRITICAL: Database file not found at root! Did you run the backfill?")
else:
    print("File found. Connecting...")

# 3. Connect using the absolute path
# We use f"sqlite:///{db_path}" (3 slashes + path)
engine = create_engine(f"sqlite:///{db_path}", echo=False)

def get_table_df(table_name: str) -> pd.DataFrame:
    with engine.connect() as conn:
        df = pd.read_sql(text(f"SELECT * FROM {table_name}"), conn)
    
    if df.empty:
        print(f"Warning: Table '{table_name}' is empty.")
    
    return df

# Usage in Notebook
df = get_table_df("crime_records")
len(df)

Looking for database at: c:\Users\nealv\OneDrive\Desktop\DA Materials\Study 2025\Python\Projects\chicago-crime\chicago_crime.db
File found. Connecting...


10000

In [11]:
import pandas as pd
import pyarrow

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    string_cols = [
        'case_number', 'block', 'iucr', 'primary_type', 
        'description', 'location_description', 'beat', 
        'district', 'community_area', 'fbi_code'
    ]
    
    return (
        df
        .pipe(lambda _df: _df.rename(columns=str.lower))
        .assign(
            # 1. Datetimes
            date=lambda x: pd.to_datetime(x['date'], errors='coerce'),
            updated_on=lambda x: pd.to_datetime(x['updated_on'], errors='coerce'),
            
            # 2. Coordinates: Float32 provides precision while saving memory
            latitude=lambda x: pd.to_numeric(x['latitude'], errors='coerce').astype('Float32'),
            longitude=lambda x: pd.to_numeric(x['longitude'], errors='coerce').astype('Float32'),
            x_coordinate=lambda x: pd.to_numeric(x['x_coordinate'], errors='coerce').astype('Float32'),
            y_coordinate=lambda x: pd.to_numeric(x['y_coordinate'], errors='coerce').astype('Float32'),
            
            # 3. Administrative IDs: Use Nullable Integers to handle missing values
            year=lambda x: pd.to_numeric(x['year'], errors='coerce').astype('Int16'),
            ward=lambda x: pd.to_numeric(x['ward'], errors='coerce').astype('Int16'),
            id=lambda x: pd.to_numeric(x['id'], errors='coerce').astype('Int64'),
            
            # 4. Nullable Booleans
            arrest=lambda x: x['arrest'].astype('boolean'),
            domestic=lambda x: x['domestic'].astype('boolean'),
            
            # 5. Optimized Strings
            **{col: lambda x, c=col: x[c].astype('string') for col in string_cols}
        )
    )

In [ ]:
import os
from datetime import datetime
import pandas as pd
from sodapy import Socrata

username = 'nealvirtudes17@gmail.com'
password = 'Stamina-Refusal-Accompany5'
token = 'SArXI6dLoYjLmaW0b5hVdpUHY'


window = datetime(2024,1,1,0,0).strftime('%Y-%m-%dT%H:%M:%S')

client = Socrata("data.cityofchicago.org",
                 f"{token}",
                 username=f"{username}",
                 password=f"{password}",
                 timeout=600 
                )

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ijzp-q8t2", where=f"date>='{window}'", limit = 100000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


https://data.cityofchicago.org/api/v3/views/igwz-8jzy/query.json


In [3]:
#community area

import os
from datetime import datetime
import pandas as pd
from sodapy import Socrata

username = 'nealvirtudes17@gmail.com'
password = 'Stamina-Refusal-Accompany5'
token = 'SArXI6dLoYjLmaW0b5hVdpUHY'


window = datetime(2024,1,1,0,0).strftime('%Y-%m-%dT%H:%M:%S')

client = Socrata("data.cityofchicago.org",
                 f"{token}",
                 username=f"{username}",
                 password=f"{password}",
                 timeout=600 
                )

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("igwz-8jzy")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df[['area_numbe','community']]



,area_numbe,community
0,1,ROGERS PARK
1,2,WEST RIDGE
2,3,UPTOWN
3,4,LINCOLN SQUARE
4,5,NORTH CENTER
...,...,...
72,73,WASHINGTON HEIGHTS
73,74,MOUNT GREENWOOD
74,75,MORGAN PARK
75,76,OHARE


In [21]:
#iucr

import os
from datetime import datetime
import pandas as pd
from sodapy import Socrata

username = 'nealvirtudes17@gmail.com'
password = 'Stamina-Refusal-Accompany5'
token = 'SArXI6dLoYjLmaW0b5hVdpUHY'


window = datetime(2024,1,1,0,0).strftime('%Y-%m-%dT%H:%M:%S')

client = Socrata("data.cityofchicago.org",
                 f"{token}",
                 username=f"{username}",
                 password=f"{password}",
                 timeout=600 
                )

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.

community_results = client.get("igwz-8jzy")

iucr_results = client.get("c7ck-438e")

ward_results = client.get("k9yb-bpqx")

beat_results = client.get("n9it-hstw")

district_results = client.get("24zt-jpfn")


# Convert to pandas DataFrame
community_results_df = pd.DataFrame.from_records(community_results)

iucr_results_df = pd.DataFrame.from_records(iucr_results)

ward_results_df = pd.DataFrame.from_records(ward_results)

beat_results_df = pd.DataFrame.from_records(beat_results)

district_results_df = pd.DataFrame.from_records(district_results)


In [ ]:
district_results_df.sort_values(by='dist_num',ascending=True)

,the_geom,dist_label,dist_num
21,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",1ST,1
20,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",10TH,10
15,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",11TH,11
14,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",12TH,12
6,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",14TH,14
24,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",15TH,15
11,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",16TH,16
0,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",17TH,17
13,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",18TH,18
4,"{'type': 'MultiPolygon', 'coordinates': [[[[-8...",19TH,19


In [ ]:
import numpy as np
from sqlalchemy.orm import Session
from sqlalchemy import insert, engine
from sqlalchemy.exc import SQLAlchemyError

def load_data_bulk(df: pd.DataFrame):
    """
    High-performance Bulk Insert for Backfilling.
    """
    if df.empty:
        print("No data to load.")
        return

    # Convert NaT/NaN to None for SQL compatibility
    clean_df = df.replace({np.nan: None})
    records = clean_df.to_dict(orient="records")

    with Session() as session:
        try:
            print(f"Inserting {len(records)} records...")
            # Core SQLAlchemy 2.0 Insert
            session.execute(insert(CrimeRecord), records)
            session.commit()
            print("Commit successful.")
        except SQLAlchemyError as e:
            session.rollback()
            print(f"Error loading data: {e}")
            raise

In [5]:
window

'2024-01-01T00:00:00'

In [10]:
results_df.info(memory_usage='deep')

<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    100000 non-null  str   
 1   case_number           100000 non-null  str   
 2   date                  100000 non-null  str   
 3   block                 100000 non-null  str   
 4   iucr                  100000 non-null  str   
 5   primary_type          100000 non-null  str   
 6   description           100000 non-null  str   
 7   location_description  99500 non-null   str   
 8   arrest                100000 non-null  bool  
 9   domestic              100000 non-null  bool  
 10  beat                  100000 non-null  str   
 11  district              100000 non-null  str   
 12  ward                  100000 non-null  str   
 13  community_area        100000 non-null  str   
 14  fbi_code              100000 non-null  str   
 15  x_coordinate          98881 n

In [19]:
clean_df = clean_data(results_df)

clean_df.info(memory_usage='deep')

<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id                    100000 non-null  Int64         
 1   case_number           100000 non-null  string        
 2   date                  100000 non-null  datetime64[us]
 3   block                 100000 non-null  string        
 4   iucr                  100000 non-null  string        
 5   primary_type          100000 non-null  string        
 6   description           100000 non-null  string        
 7   location_description  99500 non-null   string        
 8   arrest                100000 non-null  boolean       
 9   domestic              100000 non-null  boolean       
 10  beat                  100000 non-null  string        
 11  district              100000 non-null  string        
 12  ward                  100000 non-null  Int16         
 13  community_a

In [17]:
clean_df.head(3)

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,13699758,JH552112,2024-01-01,113XX S CHAMPLAIN AVE,1751,OFFENSE INVOLVING CHILDREN,CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,False,True,...,9,50,17,1182606.0,1829881.0,2024,2024-12-24 15:40:31,41.6884,-87.607117,"{'latitude': '41.688400238', 'longitude': '-87..."
1,13700521,JH553098,2024-01-01,053XX S NATCHEZ AVE,5000,OTHER OFFENSE,OTHER CRIME AGAINST PERSON,APARTMENT,False,False,...,23,56,26,1133954.0,1868394.0,2024,2024-12-25 15:40:12,41.795074,-87.784332,"{'latitude': '41.795075339', 'longitude': '-87..."
2,13700349,JH552878,2024-01-01,076XX N MARSHFIELD AVE,0266,CRIMINAL SEXUAL ASSAULT,PREDATORY,SCHOOL - PUBLIC GROUNDS,False,False,...,49,1,02,1163978.0,1950676.0,2024,2024-12-25 15:40:12,42.020283,-87.671906,"{'latitude': '42.020283994', 'longitude': '-87..."


In [23]:
clean_df['beat'].describe()

count     100000
unique       275
top         1834
freq        1194
Name: beat, dtype: object